In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import tarfile
fname = '/content/drive/MyDrive/idd-lite.tar.gz/'
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

In [5]:
import cv2
import matplotlib.pyplot as plt


In [8]:
!pip install scipy


In [9]:
import shutil
import os
import numpy as np
import json
import pandas as pd
import pprint
import tensorflow as tf
import time
from tensorflow import keras
from keras.layers import *
from keras.preprocessing import image

from glob import glob
import sys
import random

In [10]:
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers import UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from PIL import Image

In [11]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [12]:
# set maximum output limit of an ipython cell
%%javascript
IPython.OutputArea.auto_scroll_threshold = 50;

<IPython.core.display.Javascript object>

In [18]:
fine_labels_ = 'gtFine' # Folder with annotated images
leftImgsDir = 'leftImg8bit'# Folder with Png images
modes = ['train','val']  # folders with images
#data_dirs = ['Dataset','Dataset_2','Dataset_3','Dataset_4','Dataset_5','Dataset_6','Dataset_7','Dataset_8']
data_dirs = ['/content/drive/MyDrive/IDD-lite'] #main folder 
tmpAll_dir = '/content/drive/MyDrive/Anue_dataset/tmpAll' # temperoary directory to store images
#os.mkdir(tmpAll_dir) #Added




img_anno_arr = [] #array for annotated images, contains only numbers eg:571043','148237','243905','300132'


for data_dir in data_dirs: # for data in IDD_Segmentation ~ ['gtfine','leftIng8bit']
    fine_label_path = os.path.join(data_dir,fine_labels_) # IDD_Segmentation/gtFine
    leftImgPath = os.path.join(data_dir,leftImgsDir) #IDD_Segmentation/LeftImg8bit
    
    for mode in modes: # for mode in train/val
        fine_label_path_mode = os.path.join(fine_label_path,mode) # IDD_Segmentation/gtFine/mode (train/val)
        leftImgPath_mode = os.path.join(leftImgPath,mode) # IDD_Segmentation/leftImg8bit/mode (train/val)
        all_labels_dir = sorted(os.listdir(fine_label_path_mode)) #Sort all the files in fine_labels path mode
        all_imgs_dir = sorted(os.listdir(leftImgPath_mode))#Sort all the files in left_img_path_mode so that both the folders
                                                            # have the same order of images and annotated images
        common_dirs = sorted(list(set(all_labels_dir).intersection(all_imgs_dir)))#Make sure the directory is available in both 
        #gtFine and leftImgval and store it in common_dirs. If an extra directory is  there, we will not consider it
        # Directories in common_dirs are ['0','1','10','100'......]
        
        
        for com_dir in common_dirs: # take each folder like 0, then 1...
            all_fine_labels_path = os.path.join(fine_label_path_mode,com_dir)
            #all_fine_labels_path = IDD_Segmentation/gtFine/train/0
            all_imgs_path = os.path.join(leftImgPath_mode,com_dir)
            #all_imgs_path = IDD_Segmentation/leftImg8bit/train/0
            #print()
            print(com_dir)
            annots = os.listdir(all_fine_labels_path)###annots = IDD_Segemntation/gtfine/train/0/files....(If  present in images also)
            imgss = os.listdir(all_imgs_path)#####imgss = IDD_Segmentation/Img8bit/train/0/files......
            
            img_arr = [img_name[:img_name.find('_')] for img_name in imgss] 
            # IDD_Segmentation/Img8bit/train/0/005506_leftImg8bit.png is stored as 005506 in the array img_arr
            
            
            for anno in annots:
                if anno[:anno.find('_')] in img_arr: # if same annotation number is present in img_arr, them proceed
                    img_anno_arr.append(anno[:anno.find('_')]) #apend the annotated image number in array
        
                    whole_anno_path = os.path.join(all_fine_labels_path,anno)# IDD_Segmentation/Img8bit/train/0/005506_gtFine.json
                    print(whole_anno_path)
                    whole_img_name = str(anno[:anno.find('_')])+str('_leftImg8bit.png') # annotation image nmber_leftImg8bit.png
                    whole_img_path = os.path.join(all_imgs_path,whole_img_name) # IDD_Segmentation/leftImg8bit/train/0/nmber_leftImg8bit.png
                    print(whole_img_path)
                    shutil.copy(whole_anno_path,tmpAll_dir) # copy the annotated images to tempall.dir
                    shutil.copy(whole_img_path,tmpAll_dir)# copy the images to tempall.dir

In [19]:
p = sorted(os.listdir(tmpAll_dir)) #sort all images and gtfine to make them next to each other
# eg : '000035_gtFine_polygons.json', '000035_leftImg8bit.png'........
print(p)

[]


In [20]:
pk = [int(name[:name.find('_')]) for name in p]
pk = np.unique(pk) # store only unique numbers in the array
# '000035_gtFine_polygons.json', '000035_leftImg8bit.png' will be stored as 000035

In [21]:
kk = sorted(np.unique(np.array(img_anno_arr).astype(int)))
pk[pk==kk]=1
sum(pk)# number of files in the array

0